In [1]:
# Translated to .py by Monique A. Stinson
# 31.12.2016
# Adapted to PandasBiogeme by Michel Bierlaire
# Tue Oct 23 16:17:02 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable, log

df = pd.read_csv("telephone.dat",'\t')
database = db.Database("telephone",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_BM	 = Beta('ASC_BM',0,None,None,0)
ASC_SM	 = Beta('ASC_SM',0,None,None,1)
ASC_LF	 = Beta('ASC_LF',0,None,None,0)
ASC_EF	 = Beta('ASC_EF',0,None,None,0)
ASC_MF	 = Beta('ASC_MF',0,None,None,0)
B_FCOST	 = Beta('B_FCOST',0,None,None,0)
B_MCOST	 = Beta('B_MCOST',0,None,None,0)
B_USERS	 = Beta('B_USERS',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

logcostBM  = DefineVariable('logcostBM', log(cost1),database)
logcostSM  = DefineVariable('logcostSM', log(cost2),database)
logcostLF  = DefineVariable('logcostLF', log(cost3),database)
logcostEF  = DefineVariable('logcostEF', log(cost4),database)
logcostMF  = DefineVariable('logcostMF', log(cost5),database)

#Utilities
V_BM = ASC_BM + B_MCOST * logcostBM
V_SM = ASC_SM + B_MCOST * logcostSM
V_LF = ASC_LF + B_FCOST * logcostLF + B_USERS * users
V_EF = ASC_EF + B_FCOST * logcostEF + B_USERS * users
V_MF = ASC_MF + B_FCOST * logcostMF + B_USERS * users

V = {1: V_BM, 2: V_SM, 3: V_LF, 4: V_EF, 5: V_MF}
avail = {1: avail1, 2: avail2, 3: avail3, 4: avail4, 5: avail5}

# The choice model is a logit, with availability conditions
logprob = models.loglogit(V,avail,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_Tel_socioec"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


          Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_BM   -0.731    0.156   -4.69  2.7e-06         0.153        -4.77   
ASC_EF   -0.319     1.07  -0.298    0.766          1.02       -0.312   
ASC_LF  -0.0871    0.789   -0.11    0.912           0.7       -0.124   
ASC_MF    0.274    0.959   0.285    0.775          0.83         0.33   
B_FCOST   -1.79    0.315   -5.68 1.32e-08         0.286        -6.25   
B_MCOST   -1.96    0.248    -7.9 2.89e-15         0.246        -7.96   
B_USERS   0.394    0.105    3.76  0.00017         0.108         3.63   

         Rob. p-value  
ASC_BM       1.81e-06  
ASC_EF          0.755  
ASC_LF          0.901  
ASC_MF          0.742  
B_FCOST      4.23e-10  
B_MCOST      1.78e-15  
B_USERS      0.000279  
Nbr of observations: 434
LL(0) =    -560.250
LL(beta) = -468.791
rho bar square = 0.151
Output file: logit_Tel_socioec~00.html
